In [10]:
import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.functions import udf,col
from pyspark.sql.types import StringType


In [2]:
spark = SparkSession.builder.appName("Apache PySpark Final Project-Zomato").getOrCreate()

22/11/03 07:41:43 WARN Utils: Your hostname, 1011000011101110 resolves to a loopback address: 127.0.1.1; using 192.168.1.69 instead (on interface wlp3s0)
22/11/03 07:41:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/03 07:41:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [19]:
zomato_df = spark.read.csv('data/cleaned_zomato.csv',inferSchema=True,header=True)

In [20]:
zomato_df.columns

['name',
 'online_order',
 'book_table',
 'rating',
 'votes',
 'phone',
 'location',
 'type',
 'dish_liked',
 'cuisines',
 'approx_cost_two_people']

# task 1: convert "NEW" in rating to "0/5"

In [21]:
def convert(strr):
    if(strr == 'NEW'):
        return '0/5'
    else:
        return strr


In [22]:
convertUDF = udf(lambda string : convert(string),StringType())

In [23]:
zomato_df = zomato_df.withColumn("rating", convertUDF(col('rating')))

In [24]:
zomato_df.toPandas().to_csv('data/cleaned_zomato_rate.csv',index= False)

In [26]:
zomato_df.show(5)

+--------------------+------------+----------+------+-----+--------------+------------+-------------------+--------------------+--------------------+----------------------+
|                name|online_order|book_table|rating|votes|         phone|    location|               type|          dish_liked|            cuisines|approx_cost_two_people|
+--------------------+------------+----------+------+-----+--------------+------------+-------------------+--------------------+--------------------+----------------------+
|      Spice Elephant|         Yes|        No| 4.1/5|  787|  080 41714161|Banashankari|      Casual Dining|Momos, Lunch Buff...|Chinese, North In...|                   800|
|     San Churro Cafe|         Yes|        No| 3.8/5|  918|+91 9663487993|Banashankari|Cafe, Casual Dining|Churros, Cannello...|Cafe, Mexican, It...|                   800|
|Addhuri Udupi Bho...|          No|        No| 3.7/5|   88|+91 9620009302|Banashankari|        Quick Bites|         Masala Dosa|South I

# Task 2: Suggest where one can open a new restaurant